# URL Classification model using Machine Learning
### For Computer Science B.Sc. Ariel University

**By Yuval Yurzdichinsky, Matan Aviv and Roy Simanovich**

## Introduction
In this project we will build a machine learning model that will classify URLs based on if they are malicious (phishing, malware, etc.) or benign. We will use a dataset of URLs that are labeled as either malicious or benign and use it to train a model that will be able to classify new URLs.

### Install and import libraries

In [1]:
# Some importent libraries installtion via pip
%pip install scikit-learn
%pip install pandas
%pip install numpy
%pip install gensim nltk
%pip install matplotlib
%pip install python-whois
%pip install tldextract
%pip install chardet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   -------- ------------------------------ 41.0/199.4 kB 495.5 kB/s eta 0:00:01
   ------------------ -------------------- 92.2/199.4 kB 880.9 kB/s eta 0:00:01
   ---------------------------------------  194.6/199.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 199.4/199.4 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Importing the libraries

## Regular Libraries that's come with python
import pandas as pd
import numpy as np
import socket as sock
import math as m
import re
import whois as who
import tldextract as tld
from collections import Counter
from datetime import datetime

## URL Parsing
from urllib.parse import urlparse, parse_qs

## SKLearn Libraries for Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils import resample

## SKLean metrics for model evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## Tensorflow Libraries for Deep Learning
import tensorflow as tf

### Feature extraction functions
We will use the following feature extraction functions to extract features from the URLs:
1. `get_url_length`: This function returns the length of the URL.
2. `get_dot_count`: This function returns the number of dots in the URL.
3. `get_hyphen_count_in_domain`: This function returns the number of hyphens in the domain of the URL.
4. `contains_security_sensitive_words`: This function returns 1 if the URL contains security sensitive words (e.g., "login", "signin", "account", etc.) and 0 otherwise.
5. `get_directory_length`: This function returns the length of the directory part of the URL.
6. `get_sub_directory_count`: This function returns the number of subdirectories in the URL.
7. `contains_ip`: This function returns 1 if the URL contains an IP address and 0 otherwise.
8. `get_token_count_in_path`: This function returns the number of tokens in the path part of the URL.
9. `get_largest_token_length`: This function returns the length of the largest token in the path part of the URL.
10. `get_avarage_token_length`: This function returns the average length of the tokens in the path part of the URL.
11. `get_file_length`: This function returns the length of the file part of the URL.
12. `get_dot_count_in_file`: This function returns the number of dots in the file part of the URL.
13. `get_delimiter_count_in_file`: This function returns the number of delimiters in the file part of the URL.
14. `get_arguments_length`: This function returns the length of the arguments part of the URL.
15. `get_number_of_arguments`: This function returns the number of arguments in the URL.
16. `get_length_of_largest_argument_value`: This function returns the length of the largest argument value in the URL.
17. `get_max_delimiters_in_arguments`: This function returns the maximum number of delimiters in the arguments part of the URL.
18. `get_domain_features`: This function returns the following features of the domain part of the URL: length, number of dots, number of hyphens, number of digits, number of special characters, entropy, whether the domain is a subdomain, and whether the domain is an IP address.
19. `get_special_character_count`: This function returns the number of special characters in the URL.
20. `get_entropy`: This function returns the entropy of the URL.
21. `check_url_shortened`: This function returns 1 if the URL is shortened and 0 otherwise.
22. `get_port_number`: This function returns the port number of the URL, if it exists, and -1 otherwise.
23. `get_subdomain_count`: This function returns the number of subdomains in the URL.
24. `get_suspicious_tld`: This function returns 1 if the top-level domain (TLD) of the URL is suspicious (e.g., "tk", "ml", "ga", etc.) and 0 otherwise.
25. `get_numeric_ratio`: This function returns the ratio of numeric characters in the URL.
26. `get_word_count`: This function returns the number of words in the URL.
27. `get_url_is_internationalized`: This function returns 1 if the URL is internationalized and 0 otherwise.

In [7]:
# Helper function to ensure that the URL has a scheme (http:// or https://)
def ensure_scheme(url):
    if not urlparse(url).scheme:
        url = 'http://' + url
    return url

# Function to concatenate "https://" to URLs labeled with 0
def add_https(url, label):
    if label == 0:
        return "https://" + url
    else:
        return url
    
# Evaluation function for the model
def model_evaluation(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))

# Feature extraction functions

## Getting the length of the URL
def get_url_length(url):
    return len(url)

## Getting the dot count in the URL
def get_dot_count(url):
    return url.count('.')

## Check if the URL contains security sensitive words
def contains_security_sensitive_words(url):
    security_sensitive_words = [
    'login', 'password', 'admin', 'root', 'secret', 'private', 'secure', 'confidential', 
    'bank', 'creditcard', 'account', 'authentication', 'authorization', 'session', 'token', 
    'apikey', 'ssl', 'https', 'secure', 'encrypted', 'auth', 'signin', 'signup', 'verification', 
    'resetpassword', 'change-password', 'forgot-password', 'otp', '2fa', 'phishing', 'malware', 
    'virus', 'trojan', 'exploit', 'hacker', 'attack', 'security', 'vulnerable', 'injection', 
    'xss', 'csrf', 'dos', 'ddos', 'bruteforce', 'firewall', 'vpn', 'proxy', 'tor', 'security-question', 
    'privacy-policy'
]

    return int(any(word in url for word in security_sensitive_words))

## Get the directory length
def get_directory_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    return len(path)

## Get the subdirectory count
def get_sub_directory_count(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    return path.count('/') - 1

## Get the token count in the path
def get_token_count_in_path(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    tokens = path.split('/')
    return len(tokens) - 1

## Get the length of the largest token in the path
def get_largest_token_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    tokens = path.split('/')
    if tokens:
        return max(len(token) for token in tokens)
    return 0

## Get the average token length in the path
def get_average_token_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    tokens = [token for token in path.split('/') if token]
    if tokens:
        return np.mean([len(token) for token in tokens])
    return 0

## Get the file length
def get_file_length(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    filename = path.split('/')[-1]
    return len(filename)

## Get the dot count in the file
def get_dot_count_in_file(url):
    url = ensure_scheme(url)
    path = urlparse(url).path
    filename = path.split('/')[-1]
    return filename.count('.')

## Get the delimiter count in the file
def get_delimiter_count_in_file(url):
    path = urlparse(url).path
    filename = path.split('/')[-1]
    delimiters = ['.', '_', '-']
    return sum(filename.count(delimiter) for delimiter in delimiters)

## Get the arguments length
def get_arguments_length(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    return len(query)

## Get the number of arguments
def get_number_of_arguments(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    return len(parse_qs(query))

## Get the length of the largest argument value
def get_length_of_largest_argument_value(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    params = parse_qs(query)
    if params:
        return max(len(max(values, key=len)) for values in params.values())
    return 0

## Get the average argument value length
def get_max_delimiters_in_arguments(url):
    url = ensure_scheme(url)
    query = urlparse(url).query
    params = parse_qs(query)
    delimiters = ['&', '=', '-', '_']
    if params:
        return max(sum(value.count(delimiter) for delimiter in delimiters) for values in params.values() for value in values)
    return 0

## Get the hyphen count in the domain
def get_hyphen_count_in_domain(url):
    url = ensure_scheme(url)
    domain = urlparse(url).netloc
    return domain.count('-')

## Get the digit count in the domain
def contains_ip(url):
    url = ensure_scheme(url)
    domain = urlparse(url).netloc
    try:
        sock.inet_aton(domain)
        return 1
    except:
        return 0

## Get the domain length, token count, largest token length, and average token length
def get_domain_features(url):
    url = ensure_scheme(url)
    domain = urlparse(ensure_scheme(url)).netloc
    tokens = domain.split('.')
    
    # Domain Length
    domain_length = len(domain)
    
    # Count of Tokens in the Domain
    token_count = len(tokens)
    
    # Length of Largest Token in the Domain
    largest_token_length = max(len(token) for token in tokens) if tokens else 0
    
    # Average Domain Token Length
    average_token_length = sum(len(token) for token in tokens) / len(tokens) if tokens else 0
    
    return domain_length, token_count, largest_token_length, average_token_length

## Check if the URL contains special characters
def get_special_character_count(url):
    special_characters = ['@', '=', '+', '*', '?', '&', '%', '$', '#', '!']
    return sum(url.count(char) for char in special_characters)

## Get the entropy of the URL
def get_entropy(url):
    # Count the frequency of each character in the string
    freq = Counter(url)
    # Calculate the probabilities
    probs = [count / len(url) for count in freq.values()]
    # Calculate the Shannon entropy
    entropy = -sum(p * m.log(p, 2) for p in probs if p > 0)
    return entropy

## Check if the URL is shortened
def check_url_shortened(url):
    shortened_services = ['bit.ly', 'tinyurl.com', 'goo.gl', 'ow.ly', 't.co']
    url = ensure_scheme(url)
    domain = urlparse(url).netloc
    return int(domain in shortened_services)

## Get the port number, if specified
def get_port_number(url):
    url = ensure_scheme(url)
    port = urlparse(url).port
    return port if port else -1  # Return -1 if no port specified

## Get the subdomain count
def get_subdomain_count(url):
    url = ensure_scheme(url)
    domain_parts = urlparse(url).netloc.split('.')
    # Count as subdomains any parts beyond the second-level domain and TLD
    return max(0, len(domain_parts) - 2)

## Check if the URL TLD is suspicious (e.g., .xyz, .top, .loan, .win, .club)
def get_suspicious_tld(url):
    suspicious_tlds = ['xyz', 'top', 'loan', 'win', 'club']
    url = ensure_scheme(url)
    tld = urlparse(url).netloc.split('.')[-1]
    return int(tld in suspicious_tlds)

## Get the ratio of numeric characters in the URL
def get_numeric_ratio(url):
    numeric_chars = sum(c.isdigit() for c in url)
    return numeric_chars / len(url) if len(url) > 0 else 0

## Get the word count in the URL
def get_word_count(url):
    words = re.findall(r'\w+', url)
    return len(words)

## Check if the URL is internationalized (i.e., contains non-ASCII characters)
def get_url_is_internationalized(url):
    try:
        url.encode('ascii')
        return 0
    except UnicodeEncodeError:
        return 1

### Now load the dataset and do some preprocessing on it
The dataset is a CSV file with two columns: `url` and `label`. The `url` column contains the URL and the `label` column contains the label of the URL. The label is 1 if the URL is malicious and 0 if the URL is benign. Since the dataset is huge (more than 1 million rows), we will only use a small subset of it for this project (150,000 rows for each class, 300,000 rows in total).

In [8]:
# Load the dataset
df = pd.read_csv('url_dataset_updated.csv')

# Drop all duplicates from df
df = df.drop_duplicates()

# Reset the index after dropping duplicates
df = df.reset_index(drop=True)

url_column_name = 'URL'  # Replace with your actual column name

# Function to concatenate "https://" to URLs labeled with 0
def add_https(url, label):
    if label == 0:
        return "https://" + url
    else:
        return url

# Apply the function to the URL column
df[url_column_name] = df.apply(lambda row: add_https(row[url_column_name], row['Label']), axis=1)

# Separate the dataset into malicious and benign
malicious_df = df[df['Label'] == 1]
benign_df = df[df['Label'] == 0]

# Randomly sample 150,000 entries from each
malicious_sampled_df = resample(malicious_df, n_samples=150000, random_state=42)
benign_sampled_df = resample(benign_df, n_samples=150000, random_state=42)

# Combine the sampled data
balanced_df = pd.concat([malicious_sampled_df, benign_sampled_df])

# Shuffle the combined dataset to mix malicious and benign URLs
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Drop all duplicates from balanced_df
balanced_df = balanced_df.drop_duplicates()

# Reset the index after dropping duplicates
balanced_df = balanced_df.reset_index(drop=True)

# Now, balanced_df contains the balanced dataset ready for further processing

## Start testing with the options
**NOTE:** We'll split the truncated dataset into a training set and a test set (80% training, 20% test) and use the training set to train the model and the test set to evaluate the model.

### TF-IDF Vectorizer with random forest classifier, no feature extraction
We will start by using the TF-IDF vectorizer to convert the URLs into numerical features and then use a random forest classifier to classify the URLs. We will not use any feature extraction functions for now. This is for reference only, as we will use feature extraction functions and better methods later.
* **Warning:** This will take a long time to run, as the dataset is huge, and TF-IDF vectorization is a slow process. Don't run this unless you have a powerful computer.

In [5]:
# For simplicity, we'll use TF-IDF on the URLs themselves. Advanced features can be added based on URL structure and content.
#vectorizer = TfidfVectorizer()

# Prepare the data
#X = balanced_df['URL']
#y = balanced_df['Label']

# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
# Create a pipeline that first transforms the data using TfidfVectorizer then applies RandomForestClassifier
#model = make_pipeline(TfidfVectorizer(), RandomForestClassifier(n_estimators=20, random_state=42))

# Train the model
#model.fit(X_train, y_train)

# Predictions
#y_pred = model.predict(X_test)

# Evaluate the model
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
#print("Classification Report:\n", classification_report(y_test, y_pred))

### Random forest classifier, with feature extraction
Next thing we will do is use the feature extraction functions to extract features from the URLs and then use a random forest classifier to classify the URLs. We will use the same training and test sets as before.

In [13]:
# Apply feature extraction
features = balanced_df['URL'].apply(lambda x: pd.Series({
    'url_length': get_url_length(x),
    'dot_count': get_dot_count(x),
    'hyphen_count_domain': get_hyphen_count_in_domain(x),
    'security_sensitive_words': contains_security_sensitive_words(x),
    'directory_length': get_directory_length(x),
    'sub_directory_count': get_sub_directory_count(x),
    'token_count_path': get_token_count_in_path(x),
    'largest_token_length': get_largest_token_length(x),
    'average_token_length': get_average_token_length(x),
    'file_length': get_file_length(x),
    'contains_ip': contains_ip(x),
    'dot_count_in_file': get_dot_count_in_file(x),
    'delimiter_count_in_file': get_delimiter_count_in_file(x),
    'arguments_length': get_arguments_length(x),
    'number_of_arguments': get_number_of_arguments(x),
    'length_of_largest_argument_value': get_length_of_largest_argument_value(x),
    'max_delimiters_in_arguments': get_max_delimiters_in_arguments(x),
    'special_character_count': get_special_character_count(x),
    'entropy': get_entropy(x),
    'url_shortened': check_url_shortened(x),
    'port_number': get_port_number(x),
    'subdomain_count': get_subdomain_count(x),
    'suspicious_tld': get_suspicious_tld(x),
    'numeric_ratio': get_numeric_ratio(x),
    'url_is_internationalized': get_url_is_internationalized(x),
    'domain_length': get_domain_features(x)[0],
    'domain_token_count': get_domain_features(x)[1],
    'largest_domain_token_length': get_domain_features(x)[2],
    'average_domain_token_length': get_domain_features(x)[3],
    'word_count': get_word_count(x)
}))


# Concatenate original DF with features
balanced_df_with_features = pd.concat([balanced_df, features], axis=1)

# Define X and y
X = balanced_df_with_features.drop(['Label', 'URL'], axis=1)
y = balanced_df_with_features['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
# Since your features are already numerical, directly use RandomForestClassifier without TfidfVectorizer
model = RandomForestClassifier(n_estimators=50, random_state=42)

feature_selection_pipeline = Pipeline([
    ('variance_threshold', VarianceThreshold(threshold=0))
])

# Fit and transform the pipeline on the training data
X_train_transformed = feature_selection_pipeline.fit_transform(X_train, y_train)

# Transform the test data based on the fitted pipeline
X_test_transformed = feature_selection_pipeline.transform(X_test)

# Now, train your model on X_train_transformed and test on X_test_transformed
model.fit(X_train_transformed, y_train)

# Predictions
model_evaluation(model, X_test_transformed, y_test)

Accuracy: 0.945581752306841
Confusion Matrix:
 [[26911   896]
 [ 2153 26069]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.97      0.95     27807
           1       0.97      0.92      0.94     28222

    accuracy                           0.95     56029
   macro avg       0.95      0.95      0.95     56029
weighted avg       0.95      0.95      0.95     56029



## TensorFlow neural network, with feature extraction
Now let's use a neural network to classify the URLs. We will use the feature extraction functions to extract features from the URLs and then use a neural network to classify the URLs. We will use the same training and test sets as before. As the same, the training set is 80% and the test set is 20%.

### Preprocessing for Deep Learning Model

In [17]:
# Tokenization and sequence padding parameters
max_len = 100  # Adjust based on the length of the longest URL in your dataset
max_words = 60000  # Adjust based on the size of your vocabulary

# Tokenize the URLs
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(balanced_df_with_features['URL'])
sequences = tokenizer.texts_to_sequences(balanced_df_with_features['URL'])

# Pad the sequences
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

# Labels
labels = np.asarray(balanced_df_with_features['Label'])
labels = tf.keras.utils.to_categorical(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

feature_selection_pipeline = Pipeline([
    ('variance_threshold', VarianceThreshold(threshold=0))
])

# Fit and transform the pipeline on the training data
X_train_transformed = feature_selection_pipeline.fit_transform(X_train, y_train)

# Transform the test data based on the fitted pipeline
X_test_transformed = feature_selection_pipeline.transform(X_test)

### Using RNN model with feature extraction

In [18]:
# RNN model definition
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words, 32, input_length=X_train_transformed.shape[1]))
model.add(tf.keras.layers.SimpleRNN(32))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

# Define early stopping criteria
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

# Train the model with early stopping, using the CPU since the model is too simple for GPU acceleration
with tf.device('/device:CPU:0'):
    history = model.fit(X_train_transformed, y_train, epochs=10, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

# Predictions
y_pred = model.predict(X_test_transformed)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))

Epoch 1/10
1401/1401 [==============================] - 88s 62ms/step - loss: 0.1359 - acc: 0.9515 - val_loss: 0.1019 - val_acc: 0.9654
Epoch 2/10
1401/1401 [==============================] - 90s 64ms/step - loss: 0.0959 - acc: 0.9664 - val_loss: 0.0926 - val_acc: 0.9666
Epoch 3/10
1401/1401 [==============================] - 91s 65ms/step - loss: 0.0913 - acc: 0.9686 - val_loss: 0.0921 - val_acc: 0.9673
Epoch 4/10
1401/1401 [==============================] - 91s 65ms/step - loss: 0.0837 - acc: 0.9710 - val_loss: 0.0868 - val_acc: 0.9694
Epoch 5/10
1401/1401 [==============================] - 82s 58ms/step - loss: 0.0840 - acc: 0.9712 - val_loss: 0.0841 - val_acc: 0.9703
Epoch 6/10
1401/1401 [==============================] - 70s 50ms/step - loss: 0.0717 - acc: 0.9751 - val_loss: 0.0832 - val_acc: 0.9710
Epoch 7/10
1401/1401 [==============================] - 63s 45ms/step - loss: 0.0685 - acc: 0.9761 - val_loss: 0.0823 - val_acc: 0.9711
Epoch 8/10
1401/1401 [==========================

#### Optimize the TensorFlow neural network using RNN and Adam optimizer
Now let's optimize the neural network using RNN and the Adam optimizer. We will use the feature extraction functions to extract features from the URLs and then use a neural network with RNN and the Adam optimizer to classify the URLs. We will use the same training and test sets as before. Since the model now is much more complex, we'll use TensorFlow GPU acceleration to speed up the training process, if available.

In [21]:
# Tokenization and sequence padding parameters
max_len = 100  # Adjust based on the length of the longest URL in your dataset
max_words = 60000  # Adjust based on the size of your vocabulary

# Tokenize the URLs
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(balanced_df_with_features['URL'])
sequences = tokenizer.texts_to_sequences(balanced_df_with_features['URL'])

# Pad the sequences
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

# Labels
labels = np.asarray(balanced_df_with_features['Label'])
labels = tf.keras.utils.to_categorical(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

feature_selection_pipeline = Pipeline([
    ('variance_threshold', VarianceThreshold(threshold=0))
])

# Fit and transform the pipeline on the training data
X_train_transformed = feature_selection_pipeline.fit_transform(X_train, y_train)

# Transform the test data based on the fitted pipeline
X_test_transformed = feature_selection_pipeline.transform(X_test)


# Model definition
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words, 32, input_length=X_train_transformed.shape[1]))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))) # Add return_sequences if stacking LSTM layers
model.add(tf.keras.layers.Dropout(0.5))  # Adjust dropout rate as needed
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))  # Second LSTM layer, without return_sequences
model.add(tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))  # Add L2 regularization
model.add(tf.keras.layers.Dropout(0.5))  # Adjust dropout rate as needed
model.add(tf.keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])

# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

# Train the model with early stopping
# Use GPU for training, as the model is more complex and very slow to train on CPU
history = model.fit(X_train_transformed, y_train, epochs=30, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

# Predictions
y_pred = model.predict(X_test_transformed)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))

Epoch 1/30
1401/1401 [==============================] - 116s 78ms/step - loss: 0.1524 - acc: 0.9491 - val_loss: 0.0913 - val_acc: 0.9688
Epoch 2/30
1401/1401 [==============================] - 108s 77ms/step - loss: 0.0758 - acc: 0.9750 - val_loss: 0.0871 - val_acc: 0.9700
Epoch 3/30
1401/1401 [==============================] - 109s 78ms/step - loss: 0.0627 - acc: 0.9796 - val_loss: 0.0863 - val_acc: 0.9702
Epoch 4/30
1401/1401 [==============================] - 108s 77ms/step - loss: 0.0582 - acc: 0.9805 - val_loss: 0.0868 - val_acc: 0.9718
Epoch 5/30
1401/1401 [==============================] - 109s 78ms/step - loss: 0.0545 - acc: 0.9816 - val_loss: 0.0883 - val_acc: 0.9721
Epoch 6/30
1401/1401 [==============================] - 116s 83ms/step - loss: 0.0527 - acc: 0.9819 - val_loss: 0.0907 - val_acc: 0.9718
Epoch 7/30
1401/1401 [==============================] - 120s 86ms/step - loss: 0.0510 - acc: 0.9823 - val_loss: 0.0903 - val_acc: 0.9717
Epoch 8/30
1751/1751 [===================

In [12]:
# Save the trained model
model.save("RNN_and_Adam_trained_model.h5")

print("Model saved successfully.")

Model saved successfully.


#### Another optimization for the RNN model

In [14]:
# Tokenization and sequence padding parameters
max_len = 100  # Adjust based on the length of the longest URL in your dataset
max_words = 60000  # Adjust based on the size of your vocabulary

# Tokenize the URLs
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words, char_level=True)
tokenizer.fit_on_texts(balanced_df['URL'])
sequences = tokenizer.texts_to_sequences(balanced_df['URL'])

# Pad the sequences
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

# Labels
labels = np.asarray(balanced_df['Label'])
labels = tf.keras.utils.to_categorical(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# RNN model definition
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words, 32, input_length=max_len)) # Embedding layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, dropout=0.2, recurrent_dropout=0.2))) # Bidirectional LSTM layer

# Multi-layer dense network
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(32, activation='relu'))

# Dropout layer
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# Use Adam optimizer with a lower learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])

# Define early stopping criteria
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, restore_best_weights=True)

# Train the model with early stopping
# Use GPU for training, as the model is more complex and very slow to train on CPU
with tf.device('/CPU:0'):
    history = model.fit(X_train, y_train, epochs=30, batch_size=128, validation_split=0.2, callbacks=[early_stopping])

# Predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print("Confusion Matrix:\n", confusion_matrix(y_test_classes, y_pred_classes))
print("Classification Report:\n", classification_report(y_test_classes, y_pred_classes))

Epoch 1/30
1399/1399 [==============================] - 267s 187ms/step - loss: 0.2404 - acc: 0.9037 - val_loss: 0.2019 - val_acc: 0.9225
Epoch 2/30
1399/1399 [==============================] - 262s 187ms/step - loss: 0.1944 - acc: 0.9252 - val_loss: 0.1802 - val_acc: 0.9299
Epoch 3/30
1399/1399 [==============================] - 260s 186ms/step - loss: 0.1763 - acc: 0.9329 - val_loss: 0.1625 - val_acc: 0.9358
Epoch 4/30
1399/1399 [==============================] - 257s 184ms/step - loss: 0.1671 - acc: 0.9365 - val_loss: 0.1621 - val_acc: 0.9369
Epoch 5/30
1399/1399 [==============================] - 240s 172ms/step - loss: 0.1603 - acc: 0.9395 - val_loss: 0.1487 - val_acc: 0.9435
Epoch 6/30
1399/1399 [==============================] - 264s 188ms/step - loss: 0.1549 - acc: 0.9415 - val_loss: 0.1464 - val_acc: 0.9451
Epoch 7/30
1399/1399 [==============================] - 260s 186ms/step - loss: 0.1514 - acc: 0.9430 - val_loss: 0.1394 - val_acc: 0.9463
Epoch 8/30
1399/1399 [============